In [10]:
from utils import *

In [11]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [12]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["pandas", "inference-schema", "tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis_pbi.py",
)

In [13]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [14]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis-pbi",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-02 19:29:48+00:00 Creating Container Registry if not exists.
2022-01-02 19:29:49+00:00 Registering the environment.
2022-01-02 19:29:50+00:00 Use the existing image.
2022-01-02 19:29:50+00:00 Generating deployment configuration.
2022-01-02 19:29:51+00:00 Submitting deployment to compute.
2022-01-02 19:29:53+00:00 Checking the status of deployment sentiment-analysis-pbi..
2022-01-02 19:32:55+00:00 Checking the status of inference endpoint sentiment-analysis-pbi.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(service.get_logs())

2022-01-02T19:32:26,172002800+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-02T19:32:26,181220800+00:00 - rsyslog/run 
2022-01-02T19:32:26,181496600+00:00 - iot-server/run 
2022-01-02T19:32:26,225723000+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-02T19:32:26,643165200+00:00 - iot-server/finish 1 0
2022-01-02T19:32:26,657089400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (66)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 92
2022-01-02 19:32:29.718822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_8d6ea1096daefa3e7eae9e57841fbb1d/lib:/azureml-envs/azureml_8d6ea1096daefa3e7eae9e57841fb

In [16]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="sentiment-analysis-pbi")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

data = {
    "data": [
        {"query": "My ML skills are quite good."},   
        {"query": "I didn't have good experience with ML."}
    ]
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

[{"label": "POSITIVE", "score": 0.9998338222503662}, {"label": "NEGATIVE", "score": 0.9997455477714539}]


In [17]:
scoring_uri

'http://9cc99929-7203-44d5-928d-1446efe6ed7a.westus2.azurecontainer.io/score'

In [18]:
# service.delete()